# 💸 Expense Tracker - Simplilearn Submission

- **Name:** *****************
- **Email:** *****************
- **Program:** *****************  
- **Batch:** *****************
- **Date:** 2025-06-29

---


## 🎯 Objective
Build a simple CLI-based Expense Tracker to manage daily spending, categorize expenses, and compare them against a monthly budget. This was implemented as part of the ***************** by *****************.

## 🛠️ Tools & Technologies
- Python 3
- Jupyter Notebook
- Standard libraries: `csv`, `json`, `datetime`

## 📋 Features
- Add expenses with date, category, amount and description
- View sorted expense list (by date and amount)
- Set and track monthly budget
- Save data to CSV (expenses) and JSON (budgets)
- Interactive CLI with helpful prompts and input validation

## ✅ Testing
- Entered multiple expense records and budgets
- Verified file save/load and budget calculations

## 📚 Learnings
- Practiced Python file handling and user input flows
- Applied sorting and formatting logic
- Explored basic modular code structuring and CLI UX

---


### Imports

In [10]:
from datetime import datetime, date
from IPython.display import display

import csv
import json
import os
import pandas as pd

# File paths
DATA_FILE  = 'data/expenses.csv'
BUDGET_FILE = 'data/budgets.json'

### Print Helper Utility Methods

In [11]:
## PRINT HELP METHODS - Utility functions for printing text in green, blue or red and underlined ##
def printb(text):
    """Print the given text in bold blue color in the terminal.

    Args:
        text (str): The text to be printed in bold blue.
    """
    print(f"\033[1;34m{text}\033[0m")   

def printg(text):
    """Print the given text in bold green color in the terminal.

    Args:
        text (str): The text to be printed in bold green.
    """
    print(f"\033[1;32m{text}\033[0m")   

def printr(text):
    """Print the given text in bold red color in the terminal.

    Args:
        text (str): The text to be printed in bold red.
    """
    print(f"⚠️ \033[1;31m{text}\033[0m")   

def printu(text):
    """Print the given text underlined in the terminal.

    Args:
        text (str): The text to be printed with underline.
    """
    print(f"\033[4m{text}\033[0m")    
    
def print_title(text):
    """Print the given text as title with some visual markers.

    Args:
        text (str): The text to be printed as title.
    """
    print()
    print("=" * 40)
    printb(text)            
    print("=" * 40)

### 1. Add an expense

In [12]:
class ExpenseAdd:
    """
    A class to handle user input for adding an expense entry.

    Methods:
        input_expense():
            Prompts the user for all expense details including date, category, amount, and description,
            and returns an expense dictionary.

        get_date_of_expense():
            Prompts the user for the date of expense in YYYY-MM-DD format, or defaults to today's date.
            Returns a datetime.date object.

        get_expense_amount():
            Prompts the user to input the expense amount. Ensures that the value is a positive float.

        get_expense_category():
            Allows the user to select an expense category (Food or Travel) using a number or letter code.

        get_description():
            Prompts the user to enter a short description of the expense.
    """
    
    def input_expense(self):
        """Inputs the expense from user.

        Returns:
            expense: expense object like '{2025-06-27,Food,100.0,pens}'
        """
        expense_date = self.get_date_of_expense()
        category = self.get_expense_category()
        amount = self.get_expense_amount()
        description = self.get_description()

        print()
        printb("Expense details entered")
        print("-" * 40)
        printg(f"{'Date of expense:':25} {expense_date}")
        printg(f"{'Selected category:':25} {category}")
        printg(f"{'Expense amount:':25} Rs. {amount:.2f}")
        printg(f"{'Expense description:':25} {description}")
        print("-" * 40)

        expense = {
            'date': expense_date,
            'category': category,
            'amount': amount,
            'description': description
        }
        return expense
    
    def get_date_of_expense(self):
        """Prompt the user for expense category

        User can enter date in YYYY-MM-DD format or empty defaults to today's date

        Returns:
            date: A date object representing the expense date.   
        """
        user_input = input("Enter date of expense (YYYY-MM-DD), or press Enter for today: ").strip()

        if not user_input:
            return date.today()

        try:
            return datetime.strptime(user_input, "%Y-%m-%d").date()
        except ValueError:
            printr("Invalid date format. Please enter the date in YYYY-MM-DD format.")
            return self.get_date_of_expense()  # Re-prompt on error
        
    def get_expense_amount(self):
        """Prompt the user to input the expense amount.

        Ensures that the input is a positive number.

        Returns:
            float: The entered expense amount.
        """
        user_input = input("Enter expense amount ₹: ").strip()

        try:
            amount = float(user_input)
            if amount <= 0:
                raise ValueError
            return amount
        except ValueError:
            printr("Invalid amount. Please enter a positive number.")
            return self.get_expense_amount()  # Re-prompt
        
    def get_expense_category(self):
        """Prompt the user to select an expense category.

        User can enter:
            1 or F - Food
            2 or T - Travel
            (empty input) - defaults to Food

        Returns:
            str: The selected category as a string.
        """
        prompt = f"Enter expense category [1/F: Food, 2/T: Travel] (default: Food): "
        user_input = input(prompt).strip().lower()

        if user_input in ("", "1", "f"):
            return "Food"
        elif user_input in ("2", "t"):
            return "Travel"
        else:
            printr("Invalid input. Please enter 1/F or 2/T, or press Enter for default.")
            return self.get_expense_category()  # Re-prompt
        
    def get_description(self):
        """Prompt the user to input the description.

        Returns:
            string: The entered description.
        """
        return str(input("Enter the description: ").strip())

### 2. View expenses

In [13]:
class ExpenseView:
    def load_expenses(self):
        """Loads and validates expense entries from a saved CSV file.

        Reads the expenses from the predefined CSV file (DATA_FILE), validates that each
        entry contains all required fields ('date', 'category', 'amount', 'description'),
        and filters out incomplete records with a warning.

        Returns:
            list: A list of valid expense dictionaries with keys 'date', 'category', 'amount', and 'description'.
        """
        printg("Loading expenses ...")
        valid_expenses = []
        if os.path.exists(DATA_FILE):
            with open(DATA_FILE, mode='r', newline='') as file:
                reader = csv.DictReader(file)
                expenses = list(reader)
            for exp in expenses:
                if all(exp.get(k) for k in ['date', 'category', 'amount', 'description']):
                    valid_expenses.append(exp)
                else:
                    printr(f"Skipping incomplete entry: {exp}")
        
        return valid_expenses
    
    def show_expenses(self, expenses):
        """Displays the list of expenses in a tabular format.

        Converts the input list of expense dictionaries into a DataFrame,
        ensures proper formatting for date and amount columns,
        sorts by most recent date and highest amount,
        and then displays the result in a formatted table.

        Args:
            expenses (list): A list of expense dictionaries with keys 'date', 'category', 'amount', and 'description'.

        Returns:
            None
        """
        print_title("Expense View...")
        
        if not expenses:
            printr("No expenses logged yet, add your first expense!")
        else:
            df = pd.DataFrame(expenses)
            # Fix the format to work view correctly
            df['amount'] = df['amount'].astype(float)
            df['date'] = pd.to_datetime(df['date'])

            # sort by date most recent first, and then highest spend
            df = df.sort_values(by=['date', 'amount'], ascending=[False, False])

            df['amount'] = df['amount'].map('₹{:,.2f}'.format)

            display(df)

### 3. Track the Budget

In [14]:
class ExpenseTracker:
        """A class to manage and track monthly budgets and expenses.

        This class allows users to:
        - Set a monthly budget
        - Track how much has been spent for the current month
        - Load and save budget data from/to a JSON file
        - Interact via a command-line menu system

        Attributes:
            monthly_budgets (dict): A dictionary mapping 'YYYY-MM' strings to budgeted amounts.

        Methods:
            load_budgets():
                Loads monthly budgets from a JSON file, validating month format.

            write_monthly_budgets():
                Saves the current monthly budgets to a JSON file.

            track_budget(expenses):
                Calculates total spent for the current month and compares it against the budget.

            input_budget():
                Prompts the user to input or update a budget for a specific month.

            show_menu(expenses):
                Displays a menu for budget operations and handles user interaction.

            get_current_month():
                Returns the current month as a 'YYYY-MM' formatted string.

            get_month(expense_date):
                Extracts the 'YYYY-MM' string from a date or date-string.
        """
        
        def __init__(self):
            """Initializes the ExpenseTracker instance by loading previously saved budgets.
            """
            printg("Loading budget ...")
            self.monthly_budgets = self.load_budgets()
        
        def load_budgets(self):
            """Loads saved budgets from the BUDGET_FILE JSON file.

            Returns:
                dict: A dictionary of monthly budgets with 'YYYY-MM' as keys.
            """
            try:
                with open(BUDGET_FILE, "r") as file:
                    data = json.load(file)
                    saved_monthly_budgets = {}
                    for month, amount in data.items():
                        try:
                            datetime.strptime(month, "%Y-%m") # Ensure valid date format                            
                            saved_monthly_budgets[month] = amount
                        except ValueError:
                            printr(f"Skipping invalid month format: {month}")
                    return saved_monthly_budgets
                        
            except FileNotFoundError:
                return {}
            except json.JSONDecodeError:
                printr("Error reading budget file, will create new")
        
        def write_monthly_budgets(self):
            """Saves the current monthly budgets to the BUDGET_FILE JSON file.
            """
            try:
                with open(BUDGET_FILE, "w") as f:
                    json.dump(self.monthly_budgets, f, indent=4)
            except Exception as e:
                printr(f"Error saving budget file: {e}")
    
        def track_budget(self, expenses):
            """Tracks spending against the budget for the current month.

            Args:
                expenses (list): A list of expense dictionaries with 'date' and 'amount' keys.
            """
            print_title("Tracking budget view...")
            current_month = self.get_current_month()
            
            # Filter expenses for current month
            monthly_expenses = [exp for exp in expenses if self.get_month(exp['date']) == current_month]

            # Calculate total spent
            total = sum(
                float(str(exp["amount"]).strip())
                for exp in monthly_expenses
            )

            # Get budget for current month
            budget = self.monthly_budgets.get(current_month, 0.0)

            print(f"Month: {current_month}")
            print(f"Total spent: ₹{total:.2f}")
            print(f"Budget: ₹{budget:.2f}")
            remaining = budget - total
            if remaining < 0:
                printr(f"You have exceeded your budget!, remaining ₹{remaining:.2f}") 
            else:
                printg(f"You have remaining ₹{remaining:.2f} for the month")
                          
        # Function to set a monthly budget
        def input_budget(self):
            """
            Prompts the user to input a budget for a specific month (or current month by default).

            Validates the input format and updates the internal budget mapping.
            """
            print_title("Setting budget..")
            try:
                user_month = input("Enter month for the budget (YYYY-MM), or press Enter for current month: ").strip()
                if not user_month:
                    user_month = self.get_current_month()
                else:
                    datetime.strptime(user_month, "%Y-%m") # default current date
                
                month = str(user_month)
                existing_amount  = self.monthly_budgets.get(month) 
                
                if month in self.monthly_budgets:
                    amount = float(input(f"Existing budget ₹ {existing_amount:g}, Enter new amount for the {month}: ₹").strip())
                else:
                    amount = float(input(f"Enter budget amount for the {month}: ₹").strip())
                
                self.monthly_budgets[month] = amount
                printg(f"Budget of ₹{amount:g} set for {month}.")
            except ValueError:
                printr("Invalid input. Month must be in YYYY-MM format and amount must be a number.")
                return self.input_budget() # Re-prompt
        
        def show_menu(self, expenses):
            """Displays a menu of budget-related options and handles the user's selection.

            Args:
                expenses (list): A list of expense dictionaries to use for budget tracking.
            """
            print()
            printb("Enter an option for Budget tracker:")
            prompt = (
                "1/S   Set a budget for this month\n"
                "2/T   Track the budget\n"
                "3/G   Go to the previous menu\n"
            )
            user_input = input(prompt).strip().lower()

            if user_input in ("1", "s"):
                self.input_budget()
                self.write_monthly_budgets()
            elif user_input in ("", "2", "t"):
                if self.monthly_budgets.get(self.get_current_month()):                    
                    self.track_budget(expenses)
                else:
                    printr("No budget set for the current month. Please set it first.")
            elif user_input in ("3", "g", "x"):
                return
            else:
                printr("Invalid input. Please enter correct option, or press Enter for default")
                return self.show_menu()  # Re-prompt
            
        def get_current_month(self):
            """Gets the current month in 'YYYY-MM' format.

            Returns:
                str: The current month as a string.
            """
            return str(date.today().strftime("%Y-%m"))
        
        def get_month(self, expense_date):
            """Extracts the 'YYYY-MM' portion from a date or date string.

            Args:
                expense_date (date or str): The expense date, either as a datetime.date or 'YYYY-MM-DD' string.

            Returns:
                str: The month in 'YYYY-MM' format.
            """
            if isinstance(expense_date, date):
                return expense_date.strftime("%Y-%m")
            else:
                return datetime.strptime(expense_date, "%Y-%m-%d").strftime("%Y-%m")

### 4. Save expenses to the file

In [21]:
class ExpenseSave:
    """
    A class responsible for saving expense data to a CSV file.

    Methods:
        write_expenses(expenses):
            Sorts the expenses by date and amount, then writes them to a CSV file.
    """
    
    def write_expenses(self, expenses):
        """
        Writes a list of expenses to a CSV file after sorting them.

        Sort by date (desc) then amount (desc)

        Args:
            expenses (list): A list of dictionaries, each representing an expense record.
                             Each record must contain 'date', 'category', 'amount', and 'description' keys.

        Returns:
            None
        """
        if expenses:
            # Sort by date (desc) then amount (desc)
            sorted_expenses = sorted(
                expenses,
                key=lambda x: (
                    datetime.strptime(x['date'], "%Y-%m-%d").date() if isinstance(x['date'], str) else x['date'],
                    float(x['amount']) if isinstance(x['amount'], str) else x['amount']
                ),
                reverse=True
            )
        
            with open(DATA_FILE, mode='w', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=sorted_expenses[0].keys())
                writer.writeheader()
                writer.writerows(sorted_expenses)    

### Expense Manager Class

In [16]:
class ExpenseManager:
    """
    Coordinates all core components of the expense tracking application.

    This class acts as the central controller, connecting:
    - ExpenseAdd: For adding new expenses via user input
    - ExpenseView: For viewing and loading expenses
    - ExpenseSave: For persisting expenses to a file
    - ExpenseTracker: For setting and tracking budgets

    Attributes:
        expense_add (ExpenseAdd): Handles user input for expense details.
        expense_view (ExpenseView): Manages display and loading of expenses.
        expense_save (ExpenseSave): Saves expenses to persistent storage.
        expense_tracker (ExpenseTracker): Handles budgeting functionality.
        expenses (list): In-memory list of expense records.

    Methods:
        add_expense():
            Initiates user input for a new expense and asks whether to save it.

        view_expenses():
            Displays the current list of expenses in a tabular format.

        track_budget():
            Launches the budget tracking menu and allows users to set or track budgets.

        save_expenses():
            Saves the current list of expenses to file.

        exit_app():
            Saves data and cleanly exits the application.

        save_or_cancel(expense):
            Prompts the user to confirm whether to save or discard the entered expense.
    """

    def __init__(self):
        """
        Initializes the ExpenseManager by setting up component classes and loading saved expenses.
        """
        self.expense_add = ExpenseAdd()
        self.expense_view = ExpenseView()
        self.expense_save = ExpenseSave()
        self.expense_tracker = ExpenseTracker()
        self.expenses = self.expense_view.load_expenses() 

    def add_expense(self):
        """
        Prompts the user to enter a new expense and offers an option to save or cancel.
        """
        print_title("Add the expense view ... ")
        expense = self.expense_add.input_expense()
        self.save_or_cancel(expense)

    def view_expenses(self):
        """
        Displays the list of currently loaded expenses in a formatted table.
        """
        self.expense_view.show_expenses(self.expenses)

    def track_budget(self):
        """
        Opens the budget menu for setting or tracking the monthly budget.
        """
        self.expense_tracker.show_menu(self.expenses)

    def save_expenses(self):
        """
        Saves all current expense entries to the CSV file.
        """
        self.expense_save.write_expenses(self.expenses)
        printg("Data stored successfully")

    def exit_app(self):
        """
        Saves expenses and exits the application gracefully.
        """
        self.save_expenses()
        print("Exiting the Expense Tracker. Goodbye!")

    def save_or_cancel(self, expense):
        """
        Prompt the user to confirm whether to save or discard the entered expense.

        User can enter:
            1/S     Save the expense
            2/C/X   Cancel the expense
            (blank) Defaults to Save

        Args:
            expense (dict): The expense entry to potentially save.

        Returns:
            bool: True if the expense was saved, False if cancelled.
        """
        prompt = "Would you like to SAVE this expense [1/S: SAVE, 2/C: Cancel] (default: Save): "
        user_input = input(prompt).strip().lower()

        if user_input in ("", "1", "s"):
            self.expenses.append(expense)
            printg("Saved successfully.")
            return True
        elif user_input in ("2", "c", "x"):
            printg("Cancelled previous details.")     
            return False
        else:
            printr("Invalid input. Please enter 1/S, 2/C, or press Enter for default.")
            return self.save_or_cancel(expense)  # Re-prompt on invalid input     
       

### Expense App Class

In [17]:
class ExpenseApp:
    """
    Entry point for the Personal Expense Tracker application.

    This class provides a command-line interface for users to:
    - Add expenses
    - View all recorded expenses
    - Track monthly budgets
    - Save data to a persistent store
    - Exit the application gracefully

    It initializes and interacts with the ExpenseManager, which handles all core functionalities.

    Methods:
        __init__():
            Launches the interactive application loop and handles user choices.

        show_menu():
            Displays a menu of actions and returns the user's selected option.
    """

    def __init__(self):
        """
        Initializes the ExpenseApp and runs the main application loop.

        Displays a welcome message and menu options, processes user input,
        and invokes the appropriate action via the ExpenseManager.

        The application continues looping until the user selects 'Exit'.
        """
        printg("💸 Hello from your Personal Expense Tracker! 💸") 
        choice = self.show_menu()

        self.manager = ExpenseManager()   
        while choice != 5:
            if choice == 1:
                result = self.manager.add_expense()
            elif choice == 2:
                result = self.manager.view_expenses()
            elif choice == 3:
                result = self.manager.track_budget()
            elif choice == 4:
                result = self.manager.save_expenses()
            else:
                printr("Invalid choice")   
            choice = self.show_menu()

        self.manager.exit_app()
        
        return  # Using return so kernel doesn't die; use sys.exit() for standalone scripts

    def show_menu(self):
        """
        Displays the main interactive menu and prompts the user for a choice.

        Menu options include:
            1/A   Add expense
            2/V   View expenses
            3/T   Track budget
            4/S   Save expenses
            5/E/X Exit

        Returns:
            int: The selected option as an integer (1–5).
        """
        print_title("Expense tracker view..")
        printb("Enter an option to continue:")
        prompt = (
            "1/A   Add expense\n"
            "2/V   View expenses\n"
            "3/T   Track budget\n"
            "4/S   Save expenses\n"
            "5/E   Exit\n"
        )
        user_input = input(prompt).strip().lower()

        if user_input in ("", "1", "a"):
            return 1
        elif user_input in ("2", "v"):
            return 2
        elif user_input in ("3", "t"):
            return 3
        elif user_input in ("4", "s"):
            return 4
        elif user_input in ("5", "e", "x"):
            return 5
        else:
            printr("Invalid input. Please enter correct option, or press Enter for default 'Add Expense'")
            return self.show_menu()  # Re-prompt
         

### Main to run the Application from here

In [24]:
# Entry point: initialize and launch the expense tracker application
if __name__ == "__main__":
    app = ExpenseApp()
    

💸 Hello from your Personal Expense Tracker! 💸

Expense tracker view..
Enter an option to continue:


1/A   Add expense
2/V   View expenses
3/T   Track budget
4/S   Save expenses
5/E   Exit
 x


Loading budget ...
Loading expenses ...
Data stored successfully
Exiting the Expense Tracker. Goodbye!
